# Detection Model Training Notebook

1. First annotate your pictures in YOLO format (class_nr norm_x norm_y width_x_norm width_y_norm) in a tool like LabelImg
2. Make your dataset in YOLO format (for this some basic functions i used are below)
    - dataset/images/train & dataset/labels/train with matching names
    - dataset/images/val & dataset/labels/val
    - Make a data.yaml in dataset/
    - Use label 0 and 1 for red and green apples
3. Run the training of the model

In [1]:
from datetime import datetime
from ultralytics import YOLO

In [2]:
import torch
print(torch.__version__, torch.cuda.is_available(), torch.cuda.get_device_name(0))


2.5.1 True NVIDIA GeForce 940MX


In [3]:
# Load a pretrained YOLOv8n model
model = YOLO('yolov8n.pt')

In [4]:
# Extract the list of layers (nn.ModuleList)
layers = list(model.model.model)

# Find the index where the head (Detect) begins -> only freeze the backbone
detect_idx = next(i for i, m in enumerate(layers) if m.__class__.__name__ == 'Detect')
print(f"Freeze the first {detect_idx} layers (0 through {detect_idx-1})")

Freeze the first 22 layers (0 through 21)


In [5]:
def freeze_layers(model, num_layers_to_freeze=22):
    """
    Freeze the first `num_layers_to_freeze` layers of model.model.model.
    """
    layers = list(model.model.model)
    for i, layer in enumerate(layers):
        requires_grad = False if i < num_layers_to_freeze else True
        for p in layer.parameters():
            p.requires_grad = requires_grad
    # Summary
    total = sum(p.numel() for p in model.model.parameters())
    frozen = sum(p.numel() for p in model.model.parameters() if not p.requires_grad)
    print(f"Frozen parameters: {frozen}/{total} ({frozen/total*100:.1f}%)")

In [6]:
# Freeze the first 22 layers
freeze_layers(model, num_layers_to_freeze=22)

date_time_string = datetime.now().strftime("%m-%d-%H-%M")

# 3. Train with data augmentation
results = model.train(
    data='/home/jverhoog/Downloads/Basket/data.yaml',       # path to your data.yaml
    epochs=30,              # adjust as needed
    imgsz=320,              # image size
    batch=8,                # batch size
    augment=True,           # enable default augmentations
    mosaic=1.0,             # mosaic augmentation
    mixup=0.5,              # mixup augmentation
    hsv_h=0.020,            # hue jitter
    hsv_s=0.7,              # saturation jitter
    hsv_v=0.4,              # value jitter
    project='runs/train',   # where to save results
    name=f'basket_{date_time_string}'   # run name
)


Frozen parameters: 2259536/3157200 (71.6%)
New https://pypi.org/project/ultralytics/8.3.151 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.141 🚀 Python-3.10.16 torch-2.5.1 CUDA:0 (NVIDIA GeForce 940MX, 1996MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/jverhoog/Downloads/Basket/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.02, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.5, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False

train: Scanning /home/jverhoog/Downloads/Basket/labels/train... 51 images, 1 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<00:00, 720.12it/s]

train: New cache created: /home/jverhoog/Downloads/Basket/labels/train.cache


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 126.2±56.2 MB/s, size: 362.7 KB)


val: Scanning /home/jverhoog/Downloads/Basket/labels/val... 19 images, 0 backgrounds, 0 corrupt: 100%|██████████| 19/19 [00:00<00:00, 387.61it/s]

...

      30/30      0.75G     0.3938     0.8009     0.8312          3        320: 100%|██████████| 7/7 [00:02<00:00,  2.63it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  6.38it/s]

                   all         19         19      0.998      0.947      0.985      0.897



30 epochs completed in 0.031 hours.
Optimizer stripped from runs/train/basket_06-06-14-22/weights/last.pt, 6.2MB
Optimizer stripped from runs/train/basket_06-06-14-22/weights/best.pt, 6.2MB

Validating runs/train/basket_06-06-14-22/weights/best.pt...
Ultralytics 8.3.141 🚀 Python-3.10.16 torch-2.5.1 CUDA:0 (NVIDIA GeForce 940MX, 1996MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.80it/s]


                   all         19         19          1      0.945      0.985      0.901
Speed: 0.3ms preprocess, 28.5ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs/train/basket_06-06-14-22


# File rename and other help functions for yolo datasets

### Remapping class numbering in .txt files

In [7]:
# import glob

# # ─── CONFIG ────────────────────────────────────────────────────────────────────
# label_dir = "/home/jverhoog/apples_dataset/labels/train/"   # ← change to your folder
# dry_run    = False                  # ← True to preview only
# # ───────────────────────────────────────────────────────────────────────────────

# # mapping old→new
# class_map = {1: 0, 2: 1}

# # grab all .txt files (recursively)
# files = glob.glob(f"{label_dir}/**/*.txt", recursive=True)
# if not files:
#     print(f"No .txt files found in {label_dir!r}")
# else:
#     for fn in files:
#         print(f"\nProcessing: {fn}")
#         with open(fn, "r") as f:
#             lines = f.read().splitlines()

#         out_lines = []
#         changed = False

#         for i, L in enumerate(lines, start=1):
#             parts = L.split()
#             if len(parts) != 5:
#                 print(f"  ⚠️ Skipping line {i}: expected 5 elements, got {len(parts)}")
#                 continue

#             try:
#                 cls = int(parts[0])
#             except ValueError:
#                 print(f"  ❌ Line {i}: cannot parse class {parts[0]!r}")
#                 continue

#             new_cls = class_map.get(cls, cls)
#             if new_cls != cls:
#                 print(f"  ⬆️  Line {i}: class {cls} → {new_cls}")
#                 parts[0] = str(new_cls)
#                 changed = True
#             else:
#                 print(f"  ✅ Line {i}: class {cls} (no change)")

#             out_lines.append(" ".join(parts))

#         if changed:
#             if dry_run:
#                 print(f"  [dry-run] would overwrite {fn}")
#             else:
#                 with open(fn, "w") as f:
#                     f.write("\n".join(out_lines) + "\n")
#                 print(f"  ✔️  Overwritten {fn}")
#         else:
#             print(f"  ℹ️  No changes needed")


### making empty .txt files for images without annotations

In [8]:
# #!/usr/bin/env python3
# import os
# from pathlib import Path

# # ─── CONFIG ────────────────────────────────────────────────────────────────────
# base_dir    = Path('/home/jverhoog/apples_dataset/')        # ← change if your folders are elsewhere
# image_root  = base_dir / 'images'
# label_root  = base_dir / 'labels'
# subsets     = ['train', 'val'] # process these subfolders
# # ───────────────────────────────────────────────────────────────────────────────

# # allowed image extensions
# img_exts = {'.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff'}

# for subset in subsets:
#     img_dir   = image_root / subset
#     lbl_dir   = label_root / subset

#     # check image directory exists
#     if not img_dir.is_dir():
#         print(f"✖️  Image directory not found: {img_dir}")
#         continue

#     # make sure label directory exists
#     lbl_dir.mkdir(parents=True, exist_ok=True)
#     print(f"📂 Labels will be written to: {lbl_dir}")

#     # scan for image files
#     images = [p for p in img_dir.iterdir() if p.suffix.lower() in img_exts]
#     if not images:
#         print(f"⚠️  No images found in {img_dir}")
#         continue

#     for img_path in images:
#         label_path = lbl_dir / f"{img_path.stem}.txt"
#         if label_path.exists():
#             print(f"⏭ Skipping (exists): {label_path.name}")
#         else:
#             try:
#                 label_path.touch()
#                 print(f"✔️  Created: {label_path.name}")
#             except Exception as e:
#                 print(f"❌ Failed to create {label_path.name}: {e}")
